In [1]:
import sys
sys.path.insert(0, "../../code")

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
import ariadne as a

from datasets import *
from api import *
from temporary_generator import *

reading TSV: ../../data/raw/gcp/datasets/StandingSettlementInstructions/SSIPLUS_V3_MONTHLY_FULL_20200828.txt


../../code/api.py:253: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  num_payments = _num_payments,


reading TSV: ../../data/raw/gcp/datasets/StandingSettlementInstructions/BANKDIRECTORYPLUS_V3_FULL_20200828.txt


../../code/api.py:253: DtypeWarning: Columns (10,11,12,13,14,16,17,20,21,22,23,24,25,27,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  num_payments = _num_payments,


In [3]:
def assert_equals(expected, actual):
    if (expected == actual):
        return True
    else:
        raise ValueError(
            "Not equal!\n\nexpected:\n%s\n\nactual:\n%s\n" % (str(expected), str(actual))
        )

### Resolvers

In [4]:
r = DatasetsResolvers()

In [5]:
assert_equals(
    Party(
        bdp = "BD_CLIENT_Z2",
        bic = "BDABLABX840",
        name = "Z2 Corp.",
        countryCode = "SG",
        countryName = "United Kingdom",
        city = "London",
    ),
    r._client_party(),
)

True

In [6]:
assert_equals(
    [
        Party(bdp='BD0000001QJP', bic='RZOOAT2L455', name='Raiffeisenbank Region Schaerding eGen', countryCode='AT', countryName='AUSTRIA', city='Schaerding'),
        Party(bdp='BD000000RYHO', bic='BINHCN2NXXX', name='NINGBO COMMERCE BANK', countryCode='CN', countryName='CHINA', city='NINGBO'),
        Party(bdp='BD0000001H96', bic='NOLADE21STK', name='KREISSPARKASSE STADE', countryCode='DE', countryName='GERMANY', city='STADE'),
        Party(bdp='BD0000000CD2', bic='BNPADZALXXX', name='BNP PARIBAS EL DJAZAIR', countryCode='DZ', countryName='ALGERIA', city='ALGIERS'),
        Party(bdp='BD000000EVYD', bic='HNBKTWTP601', name='HUA NAN COMMERCIAL BANK, LTD.', countryCode='TW', countryName='TAIWAN', city='CHAI YI'),
    ],
    r._client_banks()
)

True

In [7]:
assert_equals(
    [
        Destination(
            party=Party(bdp='BD00000006O3', bic='BBACLBBXXXX', name='BBAC SAL', countryCode='LB', countryName='LEBANON', city='BEIRUT'),
            accounts=['0001736803002', 'USD 0001 796306-001', '0001 784202 001'],
        ),
        Destination(
            party=Party(bdp='BD0000000HUQ', bic='BSHROMRUXXX', name='SOHAR INTERNATIONAL BANK S.A.O.G', countryCode='OM', countryName='OMAN', city='MUSCAT'),
            accounts=['001020088035', '001020088026', '001020087896', '001020097024', '1020038836', '001020068518', '001020035404', '001020058867', '001020074120', '1020058867', '001020092560', '001020093392', '001020076775'],
        ),
    ],
    r._client_destinations(),
)

True

In [8]:
assert_equals(
    [
        'ANYY',
        'WHLS',
        'COPA',
        'TREA',
        'FOEX',
        'NDLF',
        'OPTI',
        'DERI',
        'MMKT',
        'LOAN',
        'SECU',
        'COLL',
        'GUAR',
        'COMM',
        'LETT',
        'TFIN',
        'DOCC',
        'CASH',
    ],
    r._asset_categories(),
)

True

In [9]:
assert_equals(
    StaticData(
        origins = [
            Party(bdp='BD0000001QJP', bic='RZOOAT2L455', name='Raiffeisenbank Region Schaerding eGen', countryCode='AT', countryName='AUSTRIA', city='Schaerding'),
            Party(bdp='BD000000RYHO', bic='BINHCN2NXXX', name='NINGBO COMMERCE BANK', countryCode='CN', countryName='CHINA', city='NINGBO'),
            Party(bdp='BD0000001H96', bic='NOLADE21STK', name='KREISSPARKASSE STADE', countryCode='DE', countryName='GERMANY', city='STADE'),
            Party(bdp='BD0000000CD2', bic='BNPADZALXXX', name='BNP PARIBAS EL DJAZAIR', countryCode='DZ', countryName='ALGERIA', city='ALGIERS'),
            Party(bdp='BD000000EVYD', bic='HNBKTWTP601', name='HUA NAN COMMERCIAL BANK, LTD.', countryCode='TW', countryName='TAIWAN', city='CHAI YI'),
        ],
        destinations = [
            Destination(
                party=Party(bdp='BD00000006O3', bic='BBACLBBXXXX', name='BBAC SAL', countryCode='LB', countryName='LEBANON', city='BEIRUT'),
                accounts=['0001736803002', 'USD 0001 796306-001', '0001 784202 001'],
            ),
            Destination(
                party=Party(bdp='BD0000000HUQ', bic='BSHROMRUXXX', name='SOHAR INTERNATIONAL BANK S.A.O.G', countryCode='OM', countryName='OMAN', city='MUSCAT'),
                accounts=['001020088035', '001020088026', '001020087896', '001020097024', '1020038836', '001020068518', '001020035404', '001020058867', '001020074120', '1020058867', '001020092560', '001020093392', '001020076775'],
            ),
        ],
        assetCategories=['ANYY', 'WHLS', 'COPA', 'TREA', 'FOEX', 'NDLF', 'OPTI', 'DERI', 'MMKT', 'LOAN', 'SECU', 'COLL', 'GUAR', 'COMM', 'LETT', 'TFIN', 'DOCC', 'CASH'],
    ),
    r.staticData(),
)

True

In [10]:
assert_equals(
    [
        Payment(originBic='BD000000RYHO', destinationBic='BD0000000HUQ', assetCategory='DERI', currency='CHF', amount=2128525),
        Payment(originBic='BD0000001QJP', destinationBic='BD0000000HUQ', assetCategory='GUAR', currency='NZD', amount=343442),
        Payment(originBic='BD000000EVYD', destinationBic='BD00000006O3', assetCategory='LOAN', currency='EUR', amount=1374073),
        Payment(originBic='BD0000001QJP', destinationBic='BD0000000HUQ', assetCategory='DOCC', currency='JPY', amount=350662),
        Payment(originBic='BD0000001H96', destinationBic='BD0000000HUQ', assetCategory='COMM', currency='NZD', amount=652946),
        Payment(originBic='BD0000001H96', destinationBic='BD00000005G6', assetCategory='FOEX', currency='GBP', amount=1216130),
        Payment(originBic='BD000000EVYD', destinationBic='BD00000005G6', assetCategory='COPA', currency='EUR', amount=780335),
        Payment(originBic='BD000000EVYD', destinationBic='BD00000006O3', assetCategory='TREA', currency='CHF', amount=779841),
        Payment(originBic='BD000000EVYD', destinationBic='BD0000000HUQ', assetCategory='LOAN', currency='CHF', amount=978510),
        Payment(originBic='BD000000RYHO', destinationBic='BD00000005G6', assetCategory='SECU', currency='NZD', amount=1459874),
    ],
    r.payments(),
)

True

### Generator

In [11]:
g = TemporaryGenerator()

In [12]:
rnd = random.Random()